In [2]:
import torch
from peft import PeftModel
import transformers

# from transformers import LLamaTokenizer, LLamaForCausalLM, GenerationConfig
from transformers import LLaMAForCausalLM, LLaMATokenizer, GenerationConfig

assert torch.cuda.is_available()
import json
import numpy as np
import math
import tqdm
import subprocess



def inference(default_model, 
              experiment_name,
              test_dataset = 't2c_concode/t2c_answers.json',
              fn_answers = "/root/CodeXGLUE/Text-Code/text-to-code/evaluator/answers.json"
             ):
    # load everything
    BASE_MODEL = default_model
    LORA_WEIGHTS = experiment_name #"tloen/alpaca-lora-7b"

    model = LLaMAForCausalLM.from_pretrained(
        BASE_MODEL,
        load_in_8bit=True,
        device_map="auto"
    )
    model = PeftModel.from_pretrained(
        model,
        LORA_WEIGHTS,
        torch_dtype=torch.float16,
    )

    tokenizer = LLaMATokenizer.from_pretrained(BASE_MODEL)
    model.config.pad_token_id = tokenizer.pad_token_id = 0  # unk
    tokenizer.padding_side = "left"
    # not sure how necessary this part is, not sure if tloen/alpaca-lora-7b was even trained with EOS and BOS tokens
    model.config.bos_token_id = 1
    model.config.eos_token_id = 2

    model.eval()

    # def generate_prompt(instruction, input):
    #     return f"Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\n{instruction}\n\n### Input:\n{input}\n\n### Response:\n"

    def generate_test_prompt(data_point, train = False):
        # To decrease expectations of results :)
        assert train == False
        # sorry about the formatting disaster gotta move fast
        if data_point["input"]:
            return f"""Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
    ### Instruction:
    {data_point["instruction"]}

    ### Input:
    {data_point["input"]}

    ### Response:
    {data_point["output"] if train else ''}"""
        else:
            return f"""Below is an instruction that describes a task. Write a response that appropriately completes the request.
    ### Instruction:
    {data_point["instruction"]}

    ### Response:
    {data_point["output"] if train else ''}"""


    lst = json.load(open(test_dataset, 'rb'))
    inputs = lst# [lst[0]]
    # instruction = 'Combine the question and answer into an image caption as succinctly as possible. Be sure to include the phrase "a photo of". Do not draw false conclusions.'
    # inputs = ['Is this a baseball game? yes', 'Is this a baseball game? no']
    prompts = [generate_test_prompt(inp) for inp in inputs]
    print(prompts[0])
    prompts = np.array(prompts)
    batch_size = 9
    res_list = []
    for ind in tqdm.tqdm(range(math.ceil(len(prompts)/batch_size))):
        current_prompts = prompts[ind*batch_size: (ind+1)*batch_size]
        print(ind*batch_size, (ind+1)*batch_size, len(current_prompts))

        tokenized_inputs = tokenizer(list(current_prompts), 
                                     padding=True, 
                                     truncation=True, 
                                     return_tensors="pt"
                                    ).to('cuda')

        generation_config = GenerationConfig(max_new_tokens=128)

        with torch.no_grad():
            full_output = model.generate(
                **tokenized_inputs,
                generation_config=generation_config
            )

        res_list.extend(tokenizer.batch_decode(full_output, skip_special_tokens=False))
    #     print(tokenizer.batch_decode(full_output, skip_special_tokens=False)[-1])


        def preprocess(s, verbose = False):
            s = s.split('### Response:\n')[-1]
            s = s.replace('\n', '  ')
            s = s.replace('<unk>', " ")
            s = ' '.join(s.split(' ')[:100])
            while '  ' in s:
                s = s.replace('  ', ' ')

            if len(s) > 0 and s[0] == ' ':
                s = s[1:]
            
            if verbose:
                print(s)
            
            return s


    predict_list = []
    for s in tqdm.tqdm(res_list):
        predict_list.append(preprocess(s))

    from datetime import datetime
    output_filename = str(datetime.now()).split('.')[0].replace(' ', '-').replace(':', '_')+'.txt'
    current_pred_name = "/root/alpaca-lora/t2c_concode/results/%s"%output_filename
    print("Save results at file", current_pred_name)

    with open(current_pred_name, 'w+') as f:
        f.write('\n'.join(predict_list))
    #     f.write('\n')

    command = f"python /root/CodeXGLUE/Text-Code/text-to-code/evaluator/evaluator.py -a {fn_answers} -p {current_pred_name}"
    result = subprocess.run(command.split(' '), 
                            stderr=subprocess.PIPE)
    result.stderr
    results = result.stderr.decode()
    print(results)
    
    return {"metric_string": results,
            "prompts_list": prompts,
            "res_list": res_list,
            "predict_list": predict_list
           }

inference(default_model = "decapoda-research/llama-7b-hf", 
          experiment_name = "t2c_concode_220428_v6"
         )





===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /opt/conda/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda117.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 8.6
CUDA SETUP: Detected CUDA version 117
CUDA SETUP: Loading binary /opt/conda/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda117.so...


/opt/conda/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('//matplotlib_inline.backend_inline'), PosixPath('module')}
  warn(msg)
/opt/conda/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: Found duplicate ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] files: {PosixPath('/usr/local/cuda/lib64/libcudart.so'), PosixPath('/usr/local/cuda/lib64/libcudart.so.11.0')}.. We'll flip a coin and try one of these, in order to fail forward.
Either way, this might cause trouble in the future:
If you get `CUDA error: invalid device function` errors, the above might be the cause and the solution is to make sure only one ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] in the paths that we search based on your env.
  warn(msg)
Overriding torch_dtype=None with `torch_dtype=torch.float16` due to requirements of `bitsandbytes` 

Loading checkpoint shards:   0%|          | 0/33 [00:00<?, ?it/s]

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
    ### Instruction:
    Generate java code
actually walks the bag to make sure the count is correct and resets the running total 

    ### Input:
     Object _current 
 int _total 
 DefaultMapBag _parent 
 Map _map 
 int _mods 
 Iterator _support 

 boolean add 
 boolean add 
 Object next 
 boolean containsAll 
 boolean containsAll 
 void clear 
 boolean isEmpty 
 boolean hasNext 
 void remove 
 boolean remove 
 boolean remove 
 Map getMap 
 int modCount 
 boolean contains 
 Iterator iterator 
 boolean removeAll 
 int size 
 boolean addAll 
 int hashCode 
 boolean equals 
 Object[] toArray 
 Object[] toArray 
 Set uniqueSet 
 void setMap 
 String toString 
 int getCount 
 List extractList 
 boolean retainAll 
 boolean retainAll

    ### Response:
    


  0%|                                                    | 0/12 [00:00<?, ?it/s]Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


...

  8%|███▋                                        | 1/12 [00:17<03:15, 17.77s/it]

9 18 9


 17%|███████▎                                    | 2/12 [01:08<06:08, 36.89s/it]

18 27 9


 17%|███████▎                                    | 2/12 [01:17<06:29, 38.99s/it]

KeyboardInterrupt



In [1]:
experiment_name = "t2c_concode_220428_v6"
# experiment_name = "t2c_concode_220428_v4"
# experiment_name = "t2c_concode_220428"
# experiment_name = "tloen/alpaca-lora-7b"

In [2]:
import torch
from peft import PeftModel
import transformers

# from transformers import LLamaTokenizer, LLamaForCausalLM, GenerationConfig
from transformers import LLaMAForCausalLM, LLaMATokenizer, GenerationConfig

assert torch.cuda.is_available()


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /opt/conda/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda117.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so.11.0
CUDA SETUP: Highest compute capability among GPUs detected: 8.6
CUDA SETUP: Detected CUDA version 117
CUDA SETUP: Loading binary /opt/conda/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda117.so...


/opt/conda/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('module'), PosixPath('//matplotlib_inline.backend_inline')}
  warn(msg)
/opt/conda/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: Found duplicate ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] files: {PosixPath('/usr/local/cuda/lib64/libcudart.so.11.0'), PosixPath('/usr/local/cuda/lib64/libcudart.so')}.. We'll flip a coin and try one of these, in order to fail forward.
Either way, this might cause trouble in the future:
If you get `CUDA error: invalid device function` errors, the above might be the cause and the solution is to make sure only one ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] in the paths that we search based on your env.
  warn(msg)


In [3]:
!ls -lah lora-alpaca/checkpoint-30
# experiment_name

ls: cannot access 'lora-alpaca/checkpoint-30': No such file or directory


In [4]:
BASE_MODEL = "decapoda-research/llama-7b-hf"
LORA_WEIGHTS = experiment_name #"tloen/alpaca-lora-7b"

model = LLaMAForCausalLM.from_pretrained(
    BASE_MODEL,
    load_in_8bit=True,
    device_map="auto"
)
model = PeftModel.from_pretrained(
    model,
    LORA_WEIGHTS,
    torch_dtype=torch.float16,
)

tokenizer = LLaMATokenizer.from_pretrained(BASE_MODEL)
model.config.pad_token_id = tokenizer.pad_token_id = 0  # unk
tokenizer.padding_side = "left"
# not sure how necessary this part is, not sure if tloen/alpaca-lora-7b was even trained with EOS and BOS tokens
model.config.bos_token_id = 1
model.config.eos_token_id = 2

model.eval()

Overriding torch_dtype=None with `torch_dtype=torch.float16` due to requirements of `bitsandbytes` to enable model loading in mixed int8. Either pass torch_dtype=torch.float16 or don't pass this argument at all to remove this warning.


Loading checkpoint shards:   0%|          | 0/33 [00:00<?, ?it/s]

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LLaMAForCausalLM(
      (model): LLaMAModel(
        (embed_tokens): Embedding(32000, 4096, padding_idx=31999)
        (layers): ModuleList(
          (0-31): 32 x LLaMADecoderLayer(
            (self_attn): LLaMAAttention(
              (q_proj): Linear8bitLt(
                in_features=4096, out_features=4096, bias=False
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=4, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=4, out_features=4096, bias=False)
                )
              )
              (k_proj): Linear8bitLt(in_features=4096, out_features=4096, bias=False)
              (v_proj): Linear8bitLt(
                in_features=4096, out_features=4096, bias=False
          

In [5]:
# model.from_pretrained("decapoda-research/llama-7b-hf", "lora-alpaca")

In [6]:
import json
import numpy as np
import math
import tqdm

# def generate_prompt(instruction, input):
#     return f"Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\n{instruction}\n\n### Input:\n{input}\n\n### Response:\n"

def generate_test_prompt(data_point, train = False):
    # To decrease expectations of results :)
    assert train == False
    # sorry about the formatting disaster gotta move fast
    if data_point["input"]:
        return f"""Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
### Instruction:
{data_point["instruction"]}

### Input:
{data_point["input"]}

### Response:
{data_point["output"] if train else ''}"""
    else:
        return f"""Below is an instruction that describes a task. Write a response that appropriately completes the request.
### Instruction:
{data_point["instruction"]}

### Response:
{data_point["output"] if train else ''}"""


lst = json.load(open('t2c_concode/t2c_answers.json', 'rb'))
inputs = lst# [lst[0]]
# instruction = 'Combine the question and answer into an image caption as succinctly as possible. Be sure to include the phrase "a photo of". Do not draw false conclusions.'
# inputs = ['Is this a baseball game? yes', 'Is this a baseball game? no']
prompts = [generate_test_prompt(inp) for inp in inputs]
print(prompts[0])
prompts = np.array(prompts)
batch_size = 9
res_list = []
for ind in tqdm.tqdm(range(math.ceil(len(prompts)/batch_size))):
    current_prompts = prompts[ind*batch_size: (ind+1)*batch_size]
    print(ind*batch_size, (ind+1)*batch_size, len(current_prompts))
    
    tokenized_inputs = tokenizer(list(current_prompts), 
                                 padding=True, 
                                 truncation=True, 
                                 return_tensors="pt"
                                ).to('cuda')

    generation_config = GenerationConfig(max_new_tokens=128)

    with torch.no_grad():
        full_output = model.generate(
            **tokenized_inputs,
            generation_config=generation_config
        )

    res_list.extend(tokenizer.batch_decode(full_output, skip_special_tokens=False))
#     print(tokenizer.batch_decode(full_output, skip_special_tokens=False)[-1])


def preprocess(s):
    s = s.split('### Response:\n')[-1]
    s = s.replace('\n', '  ')
    s = s.replace('<unk>', " ")
    s = ' '.join(s.split(' ')[:100])
    while '  ' in s:
        s = s.replace('  ', ' ')
    
    if len(s) > 0 and s[0] == ' ':
        s = s[1:]
    print(s)
    return s


predict_list = []
for s in tqdm.tqdm(res_list):
    predict_list.append(preprocess(s))
    
from datetime import datetime
output_filename = str(datetime.now()).split('.')[0].replace(' ', '-').replace(':', '_')+'.txt'
current_pred_name = "/root/alpaca-lora/t2c_concode/results/%s"%output_filename
print(current_pred_name)

with open(current_pred_name, 'w+') as f:
    f.write('\n'.join(predict_list))
#     f.write('\n')

import subprocess
command = f"python /root/CodeXGLUE/Text-Code/text-to-code/evaluator/evaluator.py -a /root/CodeXGLUE/Text-Code/text-to-code/evaluator/answers.json -p {current_pred_name}"
result = subprocess.run(command.split(' '), 
                        stderr=subprocess.PIPE)
result.stderr
results = result.stderr.decode()
print(results)


Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
### Instruction:
Generate java code
actually walks the bag to make sure the count is correct and resets the running total 

### Input:
 Object _current 
 int _total 
 DefaultMapBag _parent 
 Map _map 
 int _mods 
 Iterator _support 

 boolean add 
 boolean add 
 Object next 
 boolean containsAll 
 boolean containsAll 
 void clear 
 boolean isEmpty 
 boolean hasNext 
 void remove 
 boolean remove 
 boolean remove 
 Map getMap 
 int modCount 
 boolean contains 
 Iterator iterator 
 boolean removeAll 
 int size 
 boolean addAll 
 int hashCode 
 boolean equals 
 Object[] toArray 
 Object[] toArray 
 Set uniqueSet 
 void setMap 
 String toString 
 int getCount 
 List extractList 
 boolean retainAll 
 boolean retainAll

### Response:



  0%|                                                    | 0/12 [00:00<?, ?it/s]Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


0 9 9


  8%|███▋                                        | 1/12 [00:19<03:38, 19.87s/it]

9 18 9


...

 83%|███████████████████████████████████▊       | 10/12 [05:14<01:00, 30.25s/it]

90 99 9


 92%|███████████████████████████████████████▍   | 11/12 [05:52<00:32, 32.66s/it]

99 108 1


100%|███████████████████████████████████████████| 12/12 [05:57<00:00, 29.78s/it]


In [ ]:
0

In [8]:
def preprocess(s):
    s = s.split('### Response:\n')[-1]
    s = s.replace('\n', '  ')
    s = s.replace('<unk>', " ")
    s = ' '.join(s.split(' ')[:100])
    while '  ' in s:
        s = s.replace('  ', ' ')
    
    if len(s) > 0 and s[0] == ' ':
        s = s[1:]
    print(s)
    return s


predict_list = []
for s in tqdm.tqdm(res_list):
    predict_list.append(preprocess(s))
    
from datetime import datetime
output_filename = str(datetime.now()).split('.')[0].replace(' ', '-').replace(':', '_')+'.txt'
current_pred_name = "/root/alpaca-lora/t2c_concode/results/%s"%output_filename
print(current_pred_name)

with open(current_pred_name, 'w+') as f:
    f.write('\n'.join(predict_list))
#     f.write('\n')

import subprocess
command = f"python /root/CodeXGLUE/Text-Code/text-to-code/evaluator/evaluator.py -a /root/CodeXGLUE/Text-Code/text-to-code/evaluator/answers.json -p {current_pred_name}"
result = subprocess.run(command.split(' '), 
                        stderr=subprocess.PIPE)
result.stderr
results = result.stderr.decode()
print(results)

100%|██████████████████████████████████████| 100/100 [00:00<00:00, 29698.39it/s]

int function ( ) { int loc0 
void function ( boolean arg0 ) { fStatementsRecovery = arg0 ; } 
boolean function ( ) { return true ; } 
String function ( ) { return getIdentitiesZNode ( ) ; } 
int function ( final C 
void function ( Region arg0 ) { if ( fTagPosition == null ) { fTagPosition = arg0 ; } else { fTagPosition. setRegion ( arg0 ) ; } }
double function ( double arg0, double... arg1 ) { return factory. solve ( arg0, arg1 ) ; } 
boolean function ( ) { return hasEscapeGenerationBug ( ) ; } 

byte [ ] function ( Drawable arg0 ) { InputStream loc0 = getInputStreamFromUrl ( arg0. createInputStream ( ) ) ; byte [ ] loc1 = new byte [ loc0. getByteCount ( ) ] ; loc0. read ( loc1, 0, loc0. getByteCount ( ) ) ; return loc1 ; } 
void function ( ) { migrationDone = false ; } 
void function ( int arg0, int arg1 ) { System. out. println ( "srini_string" + arg0 + "srini_string" + arg1 ) ; } 
Dialog function ( Context arg0 
BaseColor function ( ) { return strokeColor ; } 
void function ( ) { wh

In [9]:
from datetime import datetime
output_filename = str(datetime.now()).split('.')[0].replace(' ', '-').replace(':', '_')+'.txt'
current_pred_name = "/root/alpaca-lora/t2c_concode/results/%s"%output_filename
print(current_pred_name)

with open(current_pred_name, 'w+') as f:
    f.write('\n'.join(predict_list))
#     f.write('\n')

import subprocess
command = f"python /root/CodeXGLUE/Text-Code/text-to-code/evaluator/evaluator.py -a /root/CodeXGLUE/Text-Code/text-to-code/evaluator/answers.json -p {current_pred_name}"
result = subprocess.run(command.split(' '), 
                        stderr=subprocess.PIPE)
result.stderr
results = result.stderr.decode()
print(results)

/root/alpaca-lora/t2c_concode/results/2023-04-29-18_08_01.txt
INFO:__main__:BLEU: 19.67, EM: 10.0



# len(res_list)
print(prompts[0])

In [ ]:
def preprocess(s):
    s = s.split('### Response:\n')[-1]
    s = s.replace('\n', '  ')
    s = s.replace('<unk>', " ")
    s = ' '.join(s.split(' ')[:100])
    while '  ' in s:
        s = s.replace('  ', ' ')
    
    if len(s) > 0 and s[0] == ' ':
        s = s[1:]
    print(s)
    return s


predict_list = []
for s in tqdm.tqdm(res_list):
    predict_list.append(preprocess(s))

In [ ]:
assert all([i.split("### Response:\n")[-1]=='' for i in prompts])

In [ ]:
print(lst[0])
print(prompts[0])
print(predict_list[0])

In [21]:
lst[8]['output']

'Integer function ( ) { return null == intervalQualifier ? null : intervalQualifier . getFractionalSecondPrecisionPreservingDefault ( ) ; }'

In [22]:
predict_list[8]

'Integer function ( ) { return precision ; } '

In [7]:
predict_list[9]

'byte [ ] function ( Drawable arg0 ) { return bitmapToByte ( drawableToBitmap ( arg0 ) ) ; } '

In [ ]:
from datetime import datetime
output_filename = str(datetime.now()).split('.')[0].replace(' ', '-').replace(':', '_')+'.txt'
current_pred_name = "/root/alpaca-lora/t2c_concode/results/%s"%output_filename
print(current_pred_name)

In [ ]:
with open(current_pred_name, 'w+') as f:
    f.write('\n'.join(predict_list))
#     f.write('\n')

In [ ]:
import subprocess
command = f"python /root/CodeXGLUE/Text-Code/text-to-code/evaluator/evaluator.py -a /root/CodeXGLUE/Text-Code/text-to-code/evaluator/answers.json -p {current_pred_name}"
result = subprocess.run(command.split(' '), 
                        stderr=subprocess.PIPE)
result.stderr
results = result.stderr.decode()
print(results)

In [8]:
import subprocess
command = f"python /root/CodeXGLUE/Text-Code/text-to-code/evaluator/evaluator.py -a /root/CodeXGLUE/Text-Code/text-to-code/evaluator/answers.json -p {current_pred_name}"
result = subprocess.run(command.split(' '), 
                        stderr=subprocess.PIPE)
result.stderr
results = result.stderr.decode()
print(results)

NameError: name 'current_pred_name' is not defined

In [10]:
import subprocess
command = f"python /root/CodeXGLUE/Text-Code/text-to-code/evaluator/evaluator.py -a /root/CodeXGLUE/Text-Code/text-to-code/evaluator/answers.json -p {current_pred_name}"
result = subprocess.run(command.split(' '), 
                        stderr=subprocess.PIPE)
result.stderr
results = result.stderr.decode()
print(results)

INFO:__main__:BLEU: 53.9, EM: 21.0



INFO:__main__:BLEU: 0.29, EM: 0.0



In [27]:
print(results)

INFO:__main__:BLEU: 0.27, EM: 0.0



In [66]:
model.training

False

In [4]:

# prompts = [generate_prompt(instruction=inp['instruction'], 
#                            input=inp['input']) 
#            for inp in inputs
#           ]
tokenized_inputs = tokenizer(prompts, padding=True, truncation=True, return_tensors="pt").to('cuda')

generation_config = GenerationConfig(max_new_tokens=128)

with torch.no_grad():
    full_output = model.generate(
        **tokenized_inputs,
        generation_config=generation_config
    )

print(tokenizer.batch_decode(full_output, skip_special_tokens=False)[-1])

#     # same tokenized input, but we index just into the parts for the second input
#     single_output = model.generate(
#         **{k: v[1:] for k, v in tokenized_inputs.items()},
#         generation_config=generation_config
#     )

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False


OutOfMemoryError: CUDA out of memory. Tried to allocate 3.19 GiB (GPU 0; 23.70 GiB total capacity; 13.80 GiB already allocated; 2.91 GiB free; 19.72 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [2]:
!ls t2c_concode

results  t2c_answers.json  t2c_dev.json  t2c_test.json	t2c_train.json


In [7]:
import os
import json
import pandas as pd

dct = {}
for filename in os.listdir("t2c_concode"):
    if filename.endswith('.json'):
        dct[filename] = json.load(open(f"t2c_concode/{filename}"))
    
df = {}
keys = list(dct.keys())
for key1 in keys:
    df[key1] = {}
    for key2 in keys:
        lst1 = [i['output'] for i in dct[key2]]
        lst2 = [i['output'] for i in dct[key1]]

        df[key1][key2] = len(set(lst1)&set(lst2))

pd.DataFrame(df)     

,t2c_answers.json,t2c_train.json,t2c_test.json,t2c_dev.json,t2c_1000train.json,t2c_10000train.json
t2c_answers.json,99,10,0,99,0,0
t2c_train.json,10,71262,0,114,981,9138
t2c_test.json,0,0,1,0,0,0
t2c_dev.json,99,114,0,1935,0,0
t2c_1000train.json,0,981,0,0,981,225
t2c_10000train.json,0,9138,0,0,225,9138


In [2]:
# dct.keys()
# !pip install scikit-learn

In [3]:
from sklearn.utils import shuffle
lst = set([i['output'] for i in dct['t2c_dev.json']])

lst_new = [i for i in dct['t2c_train.json'] if i['output'] not in lst]
lst_new = shuffle(lst_new)[:10000]

In [4]:
json.dump(lst_new, open("t2c_concode/t2c_10000train.json", "w+"))

In [5]:
# len(lst_new)

In [6]:
# 